## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key = g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,80.73,81,6,17.63,clear sky
1,1,Cayenne,GF,4.9333,-52.3333,82.40,89,75,14.97,light rain
2,2,Banda Aceh,ID,5.5577,95.3222,79.84,81,30,6.38,scattered clouds
3,3,Kyra,RU,49.5776,111.9753,0.64,87,92,2.35,overcast clouds
4,4,Cherskiy,RU,68.7500,161.3000,-2.72,74,65,2.17,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,80.73,81,6,17.63,clear sky
1,1,Cayenne,GF,4.9333,-52.3333,82.40,89,75,14.97,light rain
2,2,Banda Aceh,ID,5.5577,95.3222,79.84,81,30,6.38,scattered clouds
10,10,Atuona,PF,-9.8000,-139.0333,78.93,76,39,21.21,scattered clouds
17,17,Manokwari,ID,-0.8667,134.0833,78.80,83,96,2.19,overcast clouds
18,18,Pak Chong,TH,14.7080,101.4161,84.99,43,69,3.62,broken clouds
23,23,Atambua,ID,-9.1061,124.8925,75.33,90,100,3.62,overcast clouds
24,24,Rikitea,PF,-23.1203,-134.9692,77.05,73,92,10.58,overcast clouds
25,25,Neiafu,TO,-18.6500,-173.9833,77.00,94,75,12.66,moderate rain
27,27,Port Moresby,PG,-9.4431,147.1797,77.00,100,40,5.75,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count()

237


City_ID                237
City                   237
Country                235
Lat                    237
Lng                    237
Max Temp               237
Humidity               237
Cloudiness             237
Wind Speed             237
Current Description    237
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_preferred_cities_df))
clean_preferred_cities_df.head()

235


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,80.73,81,6,17.63,clear sky
1,1,Cayenne,GF,4.9333,-52.3333,82.40,89,75,14.97,light rain
2,2,Banda Aceh,ID,5.5577,95.3222,79.84,81,30,6.38,scattered clouds
10,10,Atuona,PF,-9.8000,-139.0333,78.93,76,39,21.21,scattered clouds
17,17,Manokwari,ID,-0.8667,134.0833,78.80,83,96,2.19,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,80.73,clear sky,3.0707,172.7902,
1,Cayenne,GF,82.40,light rain,4.9333,-52.3333,
2,Banda Aceh,ID,79.84,scattered clouds,5.5577,95.3222,
10,Atuona,PF,78.93,scattered clouds,-9.8000,-139.0333,
17,Manokwari,ID,78.80,overcast clouds,-0.8667,134.0833,
18,Pak Chong,TH,84.99,broken clouds,14.7080,101.4161,
23,Atambua,ID,75.33,overcast clouds,-9.1061,124.8925,
24,Rikitea,PF,77.05,overcast clouds,-23.1203,-134.9692,
25,Neiafu,TO,77.00,moderate rain,-18.6500,-173.9833,
27,Port Moresby,PG,77.00,scattered clouds,-9.4431,147.1797,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,80.73,clear sky,3.0707,172.7902,Isles Sunset Lodge
1,Cayenne,GF,82.40,light rain,4.9333,-52.3333,Hôtel Le Dronmi
2,Banda Aceh,ID,79.84,scattered clouds,5.5577,95.3222,OYO 884 Rumoh PMI Hotel
10,Atuona,PF,78.93,scattered clouds,-9.8000,-139.0333,Villa Enata
17,Manokwari,ID,78.80,overcast clouds,-0.8667,134.0833,Swiss Belhotel Manokwari


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))